In [ ]:
#########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
#from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
#%matplotlib
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
file = open('parameter.txt','w')
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
csv = "../../database/database.csv"

#fdata=pd.read_csv(csv, index_col=0)
fdata=pd.read_csv(csv, encoding = 'gbk', index_col=0)
print(fdata)
raw_data = fdata.iloc[:, 0:]

median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
#print(data)
#print(np.std(data,axis=0))
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)
#standardized_data = (data-data.mean(axis=0))/data.std(axis=0)
raw_param=standardized_data.iloc[:,0:422]
raw_power=standardized_data.iloc[:,422]
#print(raw_param)
print(raw_power)
X=raw_param.values.astype(np.float32)
y=raw_power.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15)
###########search neuron network hyperparmeter space##########
dropout_list= np.arange(0, 1, 0.1)
neurons=240
for activation in ['relu','tanh','softsign']:
    for regularizer_term in [0.0001,0.0002,0.0005,0.0008,0.001,0.002,0.005,0.0008,0.01,0.02,0.05,0.08,0.1]:
        for dropout in dropout_list:
            for epochs_number in range(1000,1200):
                for batch_size_number in range (20,2000,40):
                    for learning_rate_search in [0.001,0.002,0.005,0.01]:
                        ###########implementing hyperparameters##########
                        neurons1=neurons
                        activation1=activation
                        regularizer=keras.regularizers.l2(regularizer_term)
                        dropout_rate=dropout
                        ###########keras ANN model construction##########
                        model = Sequential() 
                        model.add(Dense(neurons1, input_dim=422, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                    bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate)) 
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                    bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                    bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate)) 
                        model.add(Dense(1, input_dim=neurons1, activation='linear'))
                        adam=tf.keras.optimizers.Adam(lr=learning_rate_search)
                        model.compile(loss='mse', optimizer=adam) 
                        #print('Training ------------')
                        ###########train the model with the training set##########
                        ###########testset has been remove before##########
                        model.fit(X_train, y_train,verbose=0, epochs=epochs_number, batch_size=batch_size_number,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
                        loss= model.evaluate(X_test, y_test)
                        predict_ann= model.predict(X_test)
                        train_ann= model.predict(X_train)
                        ###########result output##########
                        x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[42]+np.mean(data,axis=0)[422]
                        y_real_maximum_power=y_test*np.std(data,axis=0)[422]+np.mean(data,axis=0)[422]
                        x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
                        x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
                        y_real_maximum_power_series=pd.Series(y_real_maximum_power)
                        corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
                        rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
                        print('ANN,R2',corr_ann,'RMSE',rmse_val)
                        if corr_ann>0.7:
                            file.write('ANN,R2'+' '+ 'RMSE'+' '+ '\n')
                            mm1 ='%f'%corr_ann
                            mw1 ='%f'%rmse_val
                            neurons1 = '%d'%neurons
                            regularizer_term1='%f'%regularizer_term
                            dropout1='%f'%dropout
                            epochs_number1='%f'%epochs_number
                            batch_size_number1='%f'%batch_size_number
                            file.write('ANN,R2'+' '+ mm1 +' '+ 'RMSE'+' '+ mw1 +' '+neurons1+' '+regularizer_term1+' '+dropout1+' '+ epochs_number1+' '+batch_size_number1+'\n')
                            if corr_ann > max_corr_ann:
                                max_corr_ann = corr_ann
                                max_rmse_val = rmse_val
                                max_y_real_maximum_power = y_real_maximum_power
                                max_x_prediction_maximum_power_ann = x_prediction_maximum_power_ann
                                max_learning_rate_search = learning_rate_search
                                max_neurons = neurons
                                max_activation = activation
                                max_regularizer_term = regularizer_term
                                max_dropout = dropout
                                max_epochs_number = epochs_number
                                max_batch_size_number = batch_size_number
                        if 0.9<corr_ann<1:
                            break
                        else:
                            K.clear_session()
                    else:continue
                    break
                else:continue
                break
            else:continue
            break
        else:continue
        break
    else:continue
    break

###########print best hyperparameter##########
print(learning_rate_search)
print(neurons)
print(activation)
print(regularizer_term)
print(dropout)
print(epochs_number)
print(batch_size_number)
print('ANN,R2',corr_ann)
print('ANN,RMSE',rmse_val)
print(y_real_maximum_power)
###########visualization##########

            A1        A2        A3        A4       A5       A6  A7        A8  \
0     9.107689  0.256971  0.256971  0.718469  186.210  176.130  70  0.115118   
1     9.206296  0.414352  0.414352  0.557704  122.167  112.087  48  0.120884   
2     9.206296  0.414352  0.414352  0.557704  122.167  112.087  48  0.120884   
3     9.206296  0.414352  0.414352  0.557704  122.167  112.087  48  0.120884   
4    12.093131 -3.594722  0.008941  0.853625  250.275  240.195  88  0.206096   
..         ...       ...       ...       ...      ...      ...  ..       ...   
785   2.240741  1.259520  1.259520  0.579490  230.310  216.198  86 -0.017787   
786   2.178241  1.259520  1.259520  0.579490  230.310  216.198  86 -0.018361   
787   2.178241  1.259520  1.259520  0.579490  230.310  216.198  86 -0.018361   
788   2.178241  1.259520  1.259520  0.579490  230.310  216.198  86 -0.018361   
789   2.178241  1.259520  1.259520  0.579490  230.310  216.198  86 -0.018361   

           A9       A10  ...  A414  A41

D:\Anaconda\envs\tf26\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


4/4 [==============================] - 0s 2ms/step - loss: 0.7552


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10616\1765181414.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[42]+np.mean(data,axis=0)[422]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10616\1765181414.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_real_maximum_power=y_test*np.std(data,axis=0)[422]+np.mean(data,axis=0)[422]


ANN,R2 0.4907 RMSE 878.4479
